---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    # Your Code Here
    ans = []
    for G in P1_Graphs:
        degrees = G.degree()
        degree_values = sorted(set(degrees.values()))
        hist_len = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
        clust = nx.average_clustering(G)
        path_len = nx.average_shortest_path_length(G)
        if len(hist_len) > 10:
            ans.append('PA')
        elif clust > 0.1:
            ans.append('SW_L')
        else:
            ans.append('SW_H')
    return ans
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


In [5]:
G.nodes(data=True)[0]

(0, {'Department': 1, 'ManagementSalary': 0.0})

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [6]:
def salary_predictions():
    # Your Code Here
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['pagerank'] = pd.Series(nx.pagerank(G,alpha=0.8))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))

    test = df[df['ManagementSalary'].isnull()]
    train = df[~df['ManagementSalary'].isnull()]
    train['ManagementSalary'] = train['ManagementSalary'].astype(np.int64)
    X = train.loc[:,'Department':'degree']
    y = train['ManagementSalary']
    test = test.loc[:,'Department':'degree']                                   

    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score
    from sklearn.metrics import roc_curve, auc
    from sklearn.linear_model import LogisticRegression
    from sklearn.preprocessing import MinMaxScaler

    # X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    test_scaled = scaler.transform(test)

    classifier = LogisticRegression()
    clf = classifier.fit(X_scaled,y)
    # y_predict = clf.predict(X_test)
    # print('Test Scores (accuracy): ',clf.score(X_test,y_test))
    # print('Train Scores (accuracy): ',clf.score(X_train,y_train))
    # print('ROC-AOC Socres: ',roc_auc_score(y_test,y_predict))
    # y_score_lr = clf.decision_function(X_test)
    # fpr_lr, tpr_lr, _ = roc_curve(y_test, y_score_lr)
    # print('AUC: ',auc(fpr_lr, tpr_lr))
    return pd.Series(clf.predict_proba(test_scaled)[:,1],index=test.index)
salary_predictions()

/home/kartikey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/kartikey/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/kartikey/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


1       0.146906
2       0.366074
5       0.706047
8       0.152755
14      0.232214
18      0.158370
27      0.211397
30      0.229715
31      0.176142
34      0.140589
37      0.144462
40      0.201682
45      0.185736
54      0.220352
55      0.196787
60      0.177711
62      0.875591
65      0.267593
77      0.103467
79      0.136286
97      0.135243
101     0.133080
103     0.169731
108     0.145551
113     0.285901
122     0.097996
141     0.283740
142     0.642157
144     0.172608
145     0.335608
          ...   
913     0.116233
914     0.123949
915     0.067972
918     0.121170
923     0.090633
926     0.126647
931     0.123195
934     0.123394
939     0.072414
944     0.084550
945     0.089105
947     0.105138
950     0.226487
951     0.104674
953     0.098678
959     0.059745
962     0.066673
963     0.176101
968     0.109478
969     0.105887
974     0.090247
984     0.121387
987     0.131808
989     0.096751
991     0.146607
992     0.061880
994     0.081552
996     0.0718

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [7]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [10]:
def new_connections_predictions():
    
    # Your Code Here
    # jaccard = list(nx.jaccard_coefficient(G))
    # jaccard_series = pd.Series([x[2] for x in jaccard],index=[(x[0],x[1]) for x in jaccard])

    future_connections['Preferential Attachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
    future_connections['Common Neighbors'] = future_connections.index.map(lambda x: len(list(nx.common_neighbors(G, x[0], x[1]))))
    future_connections['Resource Allocation'] = [i[2] for i in nx.resource_allocation_index(G, future_connections.index)]
    future_connections['Jaccard Coefficient'] = [i[2] for i in nx.jaccard_coefficient(G, future_connections.index)]
    future_connections['Soundarjan Hopcroft'] = [i[2] for i in nx.cn_soundarajan_hopcroft(G, future_connections.index,'Department')]
    
    test = future_connections[future_connections['Future Connection'].isnull()]
    train = future_connections[~future_connections['Future Connection'].isnull()]
    train['Future Connection'] = train['Future Connection'].astype(np.int64)
    X = train.loc[:,'Preferential Attachment':'Resource Allocation']
    y = train['Future Connection']
    test = test.loc[:,'Preferential Attachment':'Resource Allocation']

    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score
    from sklearn.metrics import roc_curve, auc
    from sklearn.linear_model import LogisticRegression
    from sklearn.preprocessing import MinMaxScaler

    # X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    test_scaled = scaler.transform(test)

    classifier = LogisticRegression()
    clf = classifier.fit(X_scaled,y)
    # y_predict = clf.predict(X_test)
    # print('Test Scores (accuracy): ',clf.score(X_test,y_test))
    # print('Train Scores (accuracy): ',clf.score(X_train,y_train))
    # print('ROC-AOC Socres: ',roc_auc_score(y_test,y_predict))
    # y_score_lr = clf.decision_function(X_test)
    # fpr_lr, tpr_lr, _ = roc_curve(y_test, y_score_lr)
    # print('AUC: ',auc(fpr_lr, tpr_lr))
    return pd.Series(clf.predict_proba(test_scaled)[:,1],index=test.index)
new_connections_predictions()

/home/kartikey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/kartikey/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/kartikey/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(107, 348)    0.042310
(542, 751)    0.017157
(20, 426)     0.518413
(50, 989)     0.017286
(942, 986)    0.017425
(324, 857)    0.017268
(13, 710)     0.254570
(19, 271)     0.133621
(319, 878)    0.017331
(659, 707)    0.017171
(49, 843)     0.017371
(208, 893)    0.017202
(377, 469)    0.012395
(405, 999)    0.028723
(129, 740)    0.024425
(292, 618)    0.039211
(239, 689)    0.017340
(359, 373)    0.014249
(53, 523)     0.041041
(276, 984)    0.017364
(202, 997)    0.017391
(604, 619)    0.045885
(270, 911)    0.017337
(261, 481)    0.074435
(200, 450)    0.860879
(213, 634)    0.017117
(644, 735)    0.051508
(346, 553)    0.016798
(521, 738)    0.016189
(422, 953)    0.026179
                ...   
(672, 848)    0.017337
(28, 127)     0.905335
(202, 661)    0.016744
(54, 195)     0.999783
(295, 864)    0.017248
(814, 936)    0.017137
(839, 874)    0.017425
(139, 843)    0.017259
(461, 544)    0.015769
(68, 487)     0.015695
(622, 932)    0.017217
(504, 936)    0.022881
(479, 528) 